In [1]:
from selenium import webdriver
import urllib

Use selenium with ChromeDriver

In [2]:
driver = webdriver.Chrome()
driver.get('https://patents.google.com/')

In [3]:
# example query
queries = [['move',
  'Relocation',
  'Motion',
  'Movement',
  'Motility',
  'Travel',
  'Go',
  'Locomote',
  'Displace',
  'Proceed',
  'active',
  'Affect',
  'Impress',
  'Strike',
  'make a motion'],
 ['air', 'gas', 'atmosphere', 'Breeze', 'zephyr', 'gentle wind', 'aura'],
 ['bladeless'],
 ['fan'],
 ['f04f']]
before = '20091012'

build query parameter

In [31]:
querystr = 'q=' + '&q='.join(','.join(query) for query in queries).replace(' ', '+') + '&before=' + before
querystr

'q=move,Relocation,Motion,Movement,Motility,Travel,Go,Locomote,Displace,Proceed,active,Affect,Impress,Strike,make+a+motion&q=air,gas,atmosphere,Breeze,zephyr,gentle+wind,aura&q=bladeless&q=fan&q=f04f&before=20091012'

#### Search google patent

In [32]:
url = 'https://patents.google.com/?' + querystr
driver.get(url)

select result items extract their links

In [58]:
results = [node.get_attribute('href') for node in driver.find_elements_by_css_selector('.search-result-item a#link')]
results

['https://patents.google.com/patent/EP2191142B1/en?q=move,Relocation,Motion,Movement,Motility,Travel,Go,Locomote,Displace,Proceed,active,Affect,Impress,Strike,make+a+motion&q=air,gas,atmosphere,Breeze,zephyr,gentle+wind,aura&q=bladeless&q=fan&q=f04f&before=20091012',
 'https://patents.google.com/patent/US20100226751A1/en?q=move,Relocation,Motion,Movement,Motility,Travel,Go,Locomote,Displace,Proceed,active,Affect,Impress,Strike,make+a+motion&q=air,gas,atmosphere,Breeze,zephyr,gentle+wind,aura&q=bladeless&q=fan&q=f04f&before=20091012',
 'https://patents.google.com/patent/US8092166B2/en?q=move,Relocation,Motion,Movement,Motility,Travel,Go,Locomote,Displace,Proceed,active,Affect,Impress,Strike,make+a+motion&q=air,gas,atmosphere,Breeze,zephyr,gentle+wind,aura&q=bladeless&q=fan&q=f04f&before=20091012',
 'https://patents.google.com/patent/US20120230658A1/en?q=move,Relocation,Motion,Movement,Motility,Travel,Go,Locomote,Displace,Proceed,active,Affect,Impress,Strike,make+a+motion&q=air,gas,atmos

remove redundant part of the link

In [59]:
results = [node[:node.index('/en?q')] for node in results]
results

['https://patents.google.com/patent/EP2191142B1',
 'https://patents.google.com/patent/US20100226751A1',
 'https://patents.google.com/patent/US8092166B2',
 'https://patents.google.com/patent/US20120230658A1',
 'https://patents.google.com/patent/CN101684828B',
 'https://patents.google.com/patent/US20100226763A1',
 'https://patents.google.com/patent/US20100226749A1',
 'https://patents.google.com/patent/US20100226753A1',
 'https://patents.google.com/patent/US20100226764A1',
 'https://patents.google.com/patent/US20100226771A1']

get next page information

In [60]:
page = driver.find_element_by_css_selector('state-modifier[page="next"] > a')
if page.is_displayed():
    page.get_attribute('href')

Pack above code into a function

In [61]:
def get_search_result(url):
    driver.get(url)
    results = [node.get_attribute('href') for node in driver.find_elements_by_css_selector('.search-result-item > a#link')]
    results = [node[:node.index('/en?q')] for node in results]
    
    page = driver.find_element_by_css_selector('state-modifier[page="next"] > a')
    if page.is_displayed():
        url = page.get_attribute('href')
        # append results from next pages
        results += get_search_result(url)
    return results

In [67]:
r = get_search_result(url)
r

['https://patents.google.com/patent/EP2191142B1',
 'https://patents.google.com/patent/US20100226751A1',
 'https://patents.google.com/patent/US8092166B2',
 'https://patents.google.com/patent/US20120230658A1',
 'https://patents.google.com/patent/CN101684828B',
 'https://patents.google.com/patent/US20100226763A1',
 'https://patents.google.com/patent/US20100226749A1',
 'https://patents.google.com/patent/US20100226753A1',
 'https://patents.google.com/patent/US20100226764A1',
 'https://patents.google.com/patent/US20100226771A1']

## Get information of single patent document

In [68]:
driver.get('https://patents.google.com/patent/US20120230658A1')

In [72]:
title = driver.find_element_by_id('title').text
title

'Fan assembly'

In [73]:
patent_number = driver.find_element_by_css_selector('h2.patent-result').text
patent_number

'US20120230658A1'

In [74]:
abstract = driver.find_element_by_css_selector('div.abstract').text
abstract

'A bladeless fan assembly for creating an air current includes a nozzle mounted on a base housing a device for creating an air flow. The nozzle includes an interior passage for receiving the air flow and a mouth for emitting the air flow. The nozzle defines, and extends about, an opening through which air from outside the fan assembly is drawn by the air flow emitted from the mouth. The nozzle also includes a heater for heating the air flow upstream of the mouth.'

Test get bible information

In [75]:
from collections import defaultdict
bibinfo = defaultdict(list)
current = ''
for bibinfo_node in driver.find_elements_by_css_selector('section.knowledge-card dl > dt, dd'):
    if bibinfo_node.tag_name == 'dt':
        current = bibinfo_node.text
    elif bibinfo_node.tag_name == 'dd':
        bibinfo[current].append(bibinfo_node.text)

In [76]:
bibinfo

defaultdict(list,
            {'Application number': ['US13481268'],
             'Current Assignee': ['Dyson Technology Ltd'],
             'External links': ['USPTO',
              'USPTO Assignment',
              'Espacenet',
              'Global Dossier',
              'Discuss'],
             'Filing date': ['2012-05-25'],
             'Info': ['Patent citations (6)',
              'Cited by (59)',
              'Also published as (37)',
              'Similar documents'],
             'Inventor': ['Nicholas Gerald Fitton',
              'John Scott Sutton',
              'Peter David Gammack',
              'James Dyson',
              'John David Wallace',
              'Arran George Smith'],
             'Legal status': ['Granted'],
             'Original Assignee': ['Dyson Technology Ltd'],
             'Other versions': ['US8714937B2 (Grant)'],
             'Priority date': ['2009-03-04'],
             'Publication date': ['2012-09-13']})

In [77]:
claims = [node.text for node in driver.find_elements_by_css_selector('div.claims.patent-text > *')]
claims

['1. A bladeless fan assembly for creating an air current, the fan assembly comprising:\na device for creating an air flow;\na nozzle comprising an inner casing section, an outer casing section extending about the inner casing section, an interior passage located between the outer casing section and an annular internal surface of the inner casing section for receiving the air flow, and a mouth for emitting the air flow, the inner casing section having an annular external surface defining and extending about an opening through which air from outside the fan assembly is drawn by the air flow emitted from the mouth; and\na heater at least partially located within the nozzle and adjacent the internal surface of the inner casing section.',
 '2. The fan assembly of claim 1, wherein the heater comprises a plurality of heaters arranged about the opening.',
 '3. The fan assembly of claim 1, wherein the interior passage is annular.',
 '4. The fan assembly of claim 1, wherein the heater comprises

In [78]:
descriptions = [node.text for node in driver.find_elements_by_css_selector('div.description-line')]
descriptions

['This application is a continuation of U.S. patent application Ser. No. 12/716,780, filed Mar. 3, 2010, which claims the priority of United Kingdom Application Nos. 0903682.3, filed 4 Mar. 2009, and Ser. No. 0911178.2, filed 29 Jun. 2009, the entire contents of which are incorporated herein by reference.',
 'The present invention relates to a fan assembly. In a preferred embodiment, the present invention relates to a domestic fan, such as a tower fan, for creating a warm air current in a room, office or other domestic environment.',
 'A conventional domestic fan typically includes a set of blades or vanes mounted for rotation about an axis, and drive apparatus for rotating the set of blades to generate an air flow. The movement and circulation of the air flow creates a ‘wind chill’ or breeze and, as a result, the user experiences a cooling effect as heat is dissipated through convection and evaporation.',
 'Such fans are available in a variety of sizes and shapes. For example, a ceili